Step 5: Test Your Refactored Code

Test with valid JSON file:

In [2]:
import json
# Create a sample products.json in the current working directory
sample_data = {
    "products": [
        {
            "id": "p1",
            "name": "Water Bottle",
            "category": "Outdoors",
            "price": 12.99,
            "features": ["BPA-free", "Insulated"]
        }
    ]
}

with open("products.json", "w", encoding="utf-8") as f:
    json.dump(sample_data, f, indent=2)


In [2]:
"""
Product Description Generator - STARTER CODE (Needs Refactoring)
This code works but has many issues that need to be fixed.
"""
from dotenv import load_dotenv
import json
from openai import OpenAI
from pydantic import BaseModel, Field, validator
from typing import List, Optional
import os

load_dotenv(".env")

class Product(BaseModel):
    id: str
    name: str
    category: str
    price: float
    features: List[str] = []
    
    @validator('price')
    def price_must_be_positive(cls, v):
        if v <= 0:
            raise ValueError('Price must be positive')
        return v

def generate_product_descriptions(json_file):
    # Load JSON file
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Validate products
    products = []
    for item in data.get('products', []):
        try:
            product = Product(**item)
            products.append(product)
        except:
            pass  # Silent failure!
    
    # Generate descriptions
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    results = []
    
    for product in products:
        # Create prompt
        prompt = f"""Create a product description for:
Name: {product.name}
Category: {product.category}
Price: ${product.price}
Features: {', '.join(product.features)}

Generate a compelling product description."""
        
        # Call API
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Process response
        description = response.choices[0].message.content
        results.append({
            "product_id": product.id,
            "name": product.name,
            "description": description
        })
    
    # Save results
    with open('results.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    return results

# Usage
if __name__ == "__main__":
    generate_product_descriptions("products.json")


C:\Users\pbiai\AppData\Local\Temp\ipykernel_21788\808361677.py:21: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('price')


1 What happens if products.json doesn't exist?

In this code, with open(json_file, 'r') as f: will raise a FileNotFoundError, and the program stops because there’s no try/except around it.

2 What happens if JSON is invalid?

json.load(f) will raise a json.JSONDecodeError, and since there’s no try/except around it, the program stops with a traceback.

3 What happens if product validation fails?

Product(**item) raises a validation error, but the except: pass swallows it, so that product is silently skipped with no message.

4 What happens if API call fails?

It raises an exception (network error, auth error, rate limit, etc.) and the program stops because there’s no try/except around the API call.

5 What functions do multiple things?

The main one is generate_product_descriptions(): it does file I/O, JSON parsing, validation, prompt building, API calls, response parsing, and writing results. The validator price_must_be_positive() is single-purpose.

6 Where is code repeated that could be a helper function?

The prompt-building block inside the loop is a good candidate to extract into a helper (e.g., build_prompt(product)). If you later add more API calls or variants, the API call + response parsing could also become a helper.


Step 2: Create Helper Functions

def load_json_file(file_path: str) -> dict:
    """Load and parse JSON file with error handling."""
    # TODO: Implement with proper error handling
    # Should show WHERE error occurs if file not found or JSON invalid
    pass

def validate_product_data(product_dict: dict) -> Optional[Product]:
    """Validate product data using Pydantic."""
    # TODO: Implement with proper error handling
    # Should show WHICH fields are invalid and WHY
    pass

def create_product_prompt(product: Product) -> str:
    """Generate OpenAI prompt for product."""
    # TODO: Extract prompt creation logic
    pass

def parse_api_response(response) -> str:
    """Parse OpenAI API response."""
    # TODO: Extract response parsing logic
    pass

def format_output(product: Product, description: str) -> dict:
    """Format final output."""
    # TODO: Extract output formatting logic
    pass


1- Parsing of the json file (helper function N1)

In [ ]:
import json

def load_json_file(file_path: str) -> dict:
    """Load and parse JSON file with error handling."""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"JSON file not found: {file_path}")
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON in {file_path} (line {e.lineno}, col {e.colno}): {e.msg}")



File validation (helper function N2)

In [ ]:
from typing import Optional
from pydantic import ValidationError

def validate_product_data(product_dict: dict) -> Optional[Product]:
    """Validate product data using Pydantic."""
    try:
        return Product(**product_dict)
    except ValidationError as e:
        # Build a clear message with field -> error
        details = []
        for err in e.errors():
            field = ".".join(str(x) for x in err["loc"])
            details.append(f"{field}: {err['msg']}")
        raise ValueError("Invalid product data: " + "; ".join(details))


Prompt creation (helper function N3)

In [ ]:
def create_product_prompt(product: Product) -> str:
    """Generate OpenAI prompt for product."""
    return f"""Create a product description for:
Name: {product.name}
Category: {product.category}
Price: ${product.price}
Features: {', '.join(product.features)}

Generate a compelling product description."""


Parse the api response (helper function N4)

In [ ]:
def parse_api_response(response) -> str:
    """Parse OpenAI API response."""
    try:
        return response.choices[0].message.content
    except (AttributeError, IndexError, KeyError, TypeError):
        raise ValueError("Unexpected API response structure; missing choices/message/content")


Format Final output (helper function N5)

In [ ]:
def format_output(product: Product, description: str) -> dict:
    """Format final output."""
    return {
        "product_id": product.id,
        "name": product.name,
        "description": description,
    }


Step 3: Modularize Functions

In [ ]:
"""def load_and_validate_products(json_path: str) -> List[Product]:
    "Load JSON and validate products."
    # Uses load_json_file() and validate_product_data()
    # Should show WHERE errors occur
    pass

def generate_description(product: Product, api_client) -> str:
    "Generate description for one product using API."
    # Uses create_product_prompt() and parse_api_response()
    # Should show WHERE API errors occur
    pass

def process_products(products: List[Product], api_client) -> List[dict]:
    "Process all products and generate descriptions."
    # Orchestrates the processing
    # Should handle errors for each product
    pass

def save_results(results: List[dict], output_path: str) -> None:
    "Save results to JSON file."
    # Should show WHERE file errors occur
    pass"""


1st Function

In [ ]:
def load_products_json(json_path: str) -> dict:
    """Load JSON file only."""
    return load_json_file(json_path)

2nd Function

In [ ]:
def validate_products_list(data: dict, source: str) -> List[Product]:
    """Validate products list only."""
    products = []
    for idx, item in enumerate(data.get("products", [])):
        try:
            products.append(validate_product_data(item))
        except Exception as e:
            raise ValueError(f"Product at index {idx} invalid in {source}: {e}")
    return products


Create product prompt

In [ ]:
def create_product_prompt(product: Product) -> str:
    """Generate OpenAI prompt for product."""
    return f"""Create a product description for:
Name: {product.name}
Category: {product.category}
Price: ${product.price}
Features: {', '.join(product.features)}

Generate a compelling product description."""


Parse api response

In [ ]:
def parse_api_response(response) -> str:
    """Parse OpenAI API response."""
    try:
        return response.choices[0].message.content
    except (AttributeError, IndexError, KeyError, TypeError):
        raise ValueError("Unexpected API response structure; missing choices/message/content")


Process product

In [ ]:
def process_single_product(product: Product, api_client) -> dict:
    """Process one product and return formatted output."""
    description = generate_description(product, api_client)
    return format_output(product, description)


Process all products

In [ ]:
def process_products(products: List[Product], api_client) -> List[dict]:
    """Process all products and generate descriptions."""
    results = []
    for product in products:
        try:
            results.append(process_single_product(product, api_client))
        except Exception as e:
            # Keep going but surface which product failed
            results.append({
                "product_id": product.id,
                "name": product.name,
                "error": str(e),
            })
    return results


Save results to json file

In [ ]:
def save_results(results: List[dict], output_path: str) -> None:
    """Save results to JSON file."""
    try:
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2)
    except OSError as e:
        raise OSError(f"Failed to write results to {output_path}: {e}")


Step 4: Add Error Handling (CRITICAL)

In [ ]:
import json
import os

def load_json_file(file_path: str) -> dict:
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        error_msg = (
            "ERROR in load_json_file(): FileNotFoundError\n"
            f"  Location: File '{file_path}' not found\n"
            f"  Current directory: {os.getcwd()}\n"
            "  Suggestion: Check that the file path/location is correct"
        )
        print(error_msg)
        raise
    except json.JSONDecodeError as e:
        error_msg = (
            "ERROR in load_json_file(): JSONDecodeError\n"
            f"  Location: File '{file_path}', line {e.lineno}, column {e.colno}\n"
            f"  Reason: {e.msg}"
        )
        print(error_msg)
        raise


JSONDecodeError: Show line number and character position

In [ ]:
def load_json_file(file_path: str) -> dict:
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        ...
    except json.JSONDecodeError as e:
        ...
        error_msg = (
            "ERROR in load_json_file(): JSONDecodeError\n"
            f"  Location: File '{file_path}', line {e.lineno}, column {e.colno}\n"
            f"  Reason: {e.msg}\n"
            f"  Suggestion: Check JSON syntax at line {e.lineno}"
        )                                                   
        

Pydantic ValidationError: Show which fields are invalid and why

In [ ]:
from pydantic import ValidationError

def validate_product_data(product_dict: dict) -> Optional[Product]:
    try:
        return Product(**product_dict)
    except ValidationError as e:
        error_msg = (
            "ERROR in validate_product_data(): ValidationError\n"
            f"  Product ID: {product_dict.get('id', 'unknown')}\n"
            "  Invalid fields:\n"
        )
        for error in e.errors():
            field = ".".join(str(x) for x in error["loc"])
            error_msg += f"    - {field}: {error['msg']}\n"
        error_msg += "  Suggestion: Fix the invalid fields above"
        print(error_msg)
        return None


OpenAI API errors: Show error type, status code, and message

In [ ]:
from openai import APIError

def generate_description(product: Product, api_client) -> str:
    try:
        response = api_client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": create_product_prompt(product)}],
        )
        return parse_api_response(response)
    except APIError as e:
        error_msg = (
            "ERROR in generate_description(): APIError\n"
            f"  Product: {product.name} (ID: {product.id})\n"
            f"  Error type: {type(e).__name__}\n"
            f"  Status code: {getattr(e, 'status_code', 'N/A')}\n"
            f"  Message: {str(e)}\n"
            "  Suggestion: Check API key, rate limits, or try again later"
        )
        print(error_msg)
        raise


Network errors: Show timeout/connection details

In [ ]:
import requests

def handle_network_error(function_name: str, context: str, e: Exception) -> None:
    error_type = type(e).__name__
    error_message = str(e)
    if isinstance(e, requests.Timeout):
        error_message = f"Timeout while connecting: {e}"
    elif isinstance(e, requests.ConnectionError):
        error_message = f"Connection failed: {e}"

    error_msg = (
        f"ERROR in {function_name}(): {error_type}\n"
        f"  Location: {context}\n"
        f"  Message: {error_message}\n"
        "  Suggestion: Check internet connection, VPN/proxy, or try again later"
    )
    print(error_msg)
    raise



It detects requests.Timeout and requests.ConnectionError and replaces the message with more specific text, so you see timeout vs connection failure details in the output.